<a href="https://colab.research.google.com/github/GreihMurray/NLP-5/blob/master/anything_goes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import collections
import helper
import numpy as np
from tqdm import tqdm
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.models import Sequential

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


Original

In [ ]:
def read_train_file(file_name):
    all_data = []
    descript = 'Reading ' + file_name

    f = open(file_name, 'r', encoding='utf-8')
    full_text = f.read()

    cur_sent = []

    for line in tqdm(full_text.split('\n'), desc=descript):
        if line == '<s>':
            cur_sent = []
            continue
        if line in '()':
            continue
        if line == '</s>':
            all_data.append(cur_sent)
            continue
        else:
            cur_sent.append(line.lower())

    return all_data

Original

In [ ]:
source = read_train_file('/content/gdrive/MyDrive/Colab_Notebooks/NLP/translation/train-source.txt')
target = read_train_file('/content/gdrive/MyDrive/Colab_Notebooks/NLP/translation/train-target.txt')

Reading /content/gdrive/MyDrive/Colab_Notebooks/NLP/translation/train-source.txt: 100%|██████████| 925535/925535 [00:00<00:00, 2058536.17it/s]
Reading /content/gdrive/MyDrive/Colab_Notebooks/NLP/translation/train-target.txt: 100%|██████████| 910805/910805 [00:00<00:00, 1433167.70it/s]


Based on code from https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd

In [ ]:
source_counter = collections.Counter([word for sentence in source for word in sentence])
target_counter = collections.Counter([word for sentence in target for word in sentence])

print('{} Source words.'.format(len([word for sentence in source for word in sentence])))
print('{} unique source words.'.format(len(source_counter)))
print('10 Most common words in the source dataset:')
print('"' + '" "'.join(list(zip(*source_counter.most_common(10)))[0]) + '"')
print()
print('{} Target words.'.format(len([word for sentence in target for word in sentence])))
print('{} unique target words.'.format(len(target_counter)))
print('10 Most common words in the target dataset:')
print('"' + '" "'.join(list(zip(*target_counter.most_common(10)))[0]) + '"')

834933 Source words.
31035 unique source words.
10 Most common words in the source dataset:
"." "a" "an" "," "agus" "ar" """ "bhí" "na" "sé"

820239 Target words.
25735 unique target words.
10 Most common words in the target dataset:
"." "a" "an" "," "agus" "ar" "bhí" """ "ag" "sé"


Based on code from https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd

In [ ]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    x_tk = Tokenizer(char_level = False)
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk

Based on code from https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd

In [ ]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen = length, padding = 'post')

Based on code from https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd

In [ ]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

Based on code from https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd

In [ ]:
pre_source, pre_target, source_token, target_token =\
    preprocess(source, target)
    
max_source_length = pre_source.shape[1]
max_target_length = pre_target.shape[1]
source_vocab_size = len(source_token.word_index)
target_vocab_size = len(target_token.word_index)

print('Data Preprocessed')
print("Max source sentence length:", max_source_length)
print("Max target sentence length:", max_target_length)
print("Source vocabulary size:", source_vocab_size)
print("Target vocabulary size:", target_vocab_size)

Data Preprocessed
Max source sentence length: 231
Max target sentence length: 221
Source vocabulary size: 31035
Target vocabulary size: 25735


Based on code from https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd

In [ ]:
def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = ''

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

Based on code from https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd

In [ ]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    # TODO: Build the layers
    learning_rate = 1e-3
    input_seq = Input(input_shape[1:])
    rnn = GRU(64, return_sequences = True)(input_seq)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn)
    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    
    return model

Based on code from https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd

In [ ]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    # TODO: Implement
    learning_rate = 1e-3
    rnn = GRU(64, return_sequences=True, activation="tanh")
    
    embedding = Embedding(french_vocab_size, 64, input_length=input_shape[1]) 
    logits = TimeDistributed(Dense(french_vocab_size, activation="softmax"))
    
    model = Sequential()
    #em can only be used in first layer --> Keras Documentation
    model.add(embedding)
    model.add(rnn)
    model.add(logits)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

Based on code from https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd

In [ ]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    learning_rate = 1e-3
    model = Sequential()
    model.add(Bidirectional(GRU(128, return_sequences = True, dropout = 0.1), 
                           input_shape = input_shape[1:]))
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    return model

Based on code from https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd

In [ ]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement
    learning_rate = 1e-3
    model = Sequential()
    model.add(GRU(128, input_shape = input_shape[1:], return_sequences = False))
    model.add(RepeatVector(output_sequence_length))
    model.add(GRU(128, return_sequences = True))
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
    
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    return model


Based on code from https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd

In [ ]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    model = Sequential()
    model.add(Embedding(input_dim=english_vocab_size,output_dim=128,input_length=input_shape[1]))
    model.add(Bidirectional(GRU(256,return_sequences=False)))
    model.add(RepeatVector(output_sequence_length))
    model.add(Bidirectional(GRU(256,return_sequences=True)))
    model.add(TimeDistributed(Dense(french_vocab_size,activation='softmax')))
    learning_rate = 0.005
    
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    
    return model

Based on code from https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd

In [ ]:
# Train Simple RNN

tmp_x = pad(pre_source, max_target_length)
tmp_x = tmp_x.reshape((-1, pre_target.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_target_length,
    source_vocab_size,
    target_vocab_size)
simple_rnn_model.fit(tmp_x, pre_target, batch_size=32, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], target_token))

Epoch 1/10
1130/1130 [==============================] - 161s 141ms/step - loss: 0.9704 - accuracy: 0.9192 - val_loss: nan - val_accuracy: 0.9287
Epoch 2/10
1130/1130 [==============================] - 163s 144ms/step - loss: 0.5466 - accuracy: 0.9205 - val_loss: nan - val_accuracy: 0.9290
Epoch 3/10
1130/1130 [==============================] - 163s 144ms/step - loss: 0.5401 - accuracy: 0.9207 - val_loss: nan - val_accuracy: 0.9290
Epoch 4/10
1130/1130 [==============================] - 162s 144ms/step - loss: 0.5366 - accuracy: 0.9209 - val_loss: nan - val_accuracy: 0.9291
Epoch 5/10
1130/1130 [==============================] - 162s 144ms/step - loss: 0.5340 - accuracy: 0.9210 - val_loss: nan - val_accuracy: 0.9291
Epoch 6/10
1130/1130 [==============================] - 162s 144ms/step - loss: 0.5317 - accuracy: 0.9211 - val_loss: nan - val_accuracy: 0.9291
Epoch 7/10
1130/1130 [==============================] - 162s 144ms/step - loss: 0.5298 - accuracy: 0.9210 - val_loss: nan - val_ac

Original

In [ ]:
simple_rnn_model.save('/content/gdrive/MyDrive/Colab_Notebooks/NLP/translation/simple_rnn')

Based on code from https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd

In [ ]:
# Train bi-directional

tmp_x = pad(pre_source, pre_target.shape[1])
tmp_x = tmp_x.reshape((-1, pre_target.shape[-2], 1))

bidi_model = bd_model(
    tmp_x.shape,
    pre_target.shape[1],
    len(source_token.word_index)+1,
    len(target_token.word_index)+1)


bidi_model.fit(tmp_x, pre_target, batch_size=32, epochs=20, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(bidi_model.predict(tmp_x[:1])[0], target_token))

Epoch 1/20
1130/1130 [==============================] - 335s 294ms/step - loss: 0.7390 - accuracy: 0.9194 - val_loss: 0.4859 - val_accuracy: 0.9286
Epoch 2/20
1130/1130 [==============================] - 332s 294ms/step - loss: 0.5451 - accuracy: 0.9205 - val_loss: 0.4838 - val_accuracy: 0.9291
Epoch 3/20
1130/1130 [==============================] - 333s 294ms/step - loss: 0.5402 - accuracy: 0.9207 - val_loss: 0.4811 - val_accuracy: 0.9293
Epoch 4/20
1130/1130 [==============================] - 333s 294ms/step - loss: 0.5340 - accuracy: 0.9211 - val_loss: 0.4775 - val_accuracy: 0.9297
Epoch 5/20
1130/1130 [==============================] - 333s 294ms/step - loss: 0.5272 - accuracy: 0.9216 - val_loss: 0.4751 - val_accuracy: 0.9296
Epoch 6/20
1130/1130 [==============================] - 333s 295ms/step - loss: 0.5210 - accuracy: 0.9219 - val_loss: 0.4719 - val_accuracy: 0.9302
Epoch 7/20
1130/1130 [==============================] - 333s 294ms/step - loss: 0.5146 - accuracy: 0.9223 - val_

Original

In [ ]:
bidi_model.save('/content/gdrive/MyDrive/Colab_Notebooks/NLP/translation/bidirection')

Based on code from https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd

In [ ]:
# Train embedding model

tmp_x = pad(pre_source, max_target_length)
tmp_x = tmp_x.reshape((-1, pre_target.shape[-2]))

# TODO: Train the neural network

embeded_model = embed_model(
    tmp_x.shape,
    max_target_length,
    source_vocab_size,
    target_vocab_size)

embeded_model.fit(tmp_x, pre_target, batch_size=32, epochs=10, validation_split=0.2)


# TODO: Print prediction(s)
print(logits_to_text(embeded_model.predict(tmp_x[:1])[0], target_token))

Epoch 1/10
1130/1130 [==============================] - 270s 234ms/step - loss: 0.9987 - accuracy: 0.9153 - val_loss: nan - val_accuracy: 0.9280
Epoch 2/10
1130/1130 [==============================] - 265s 234ms/step - loss: 0.5450 - accuracy: 0.9201 - val_loss: nan - val_accuracy: 0.9294
Epoch 3/10
1130/1130 [==============================] - 265s 235ms/step - loss: 0.5372 - accuracy: 0.9215 - val_loss: nan - val_accuracy: 0.9296
Epoch 4/10
1130/1130 [==============================] - 265s 235ms/step - loss: 0.5310 - accuracy: 0.9221 - val_loss: nan - val_accuracy: 0.9298
Epoch 5/10
1130/1130 [==============================] - 265s 235ms/step - loss: 0.5252 - accuracy: 0.9225 - val_loss: nan - val_accuracy: 0.9295
Epoch 6/10
1130/1130 [==============================] - 265s 235ms/step - loss: 0.5208 - accuracy: 0.9228 - val_loss: nan - val_accuracy: 0.9299
Epoch 7/10
1130/1130 [==============================] - 265s 235ms/step - loss: 0.5167 - accuracy: 0.9230 - val_loss: nan - val_ac

Original

In [ ]:
embeded_model.save('/content/gdrive/MyDrive/Colab_Notebooks/NLP/translation/embedding')

Based on code from https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd

Original

In [ ]:
# Train encode/decode model

tmp_x = pad(pre_source)
tmp_x = tmp_x.reshape((-1, pre_target.shape[-2]))

encodeco_model = encdec_model(
    tmp_x.shape,
    pre_source.shape[1],
    len(source_token.word_index)+1,
    len(target_token.word_index)+1)

encodeco_model.fit(tmp_x, pre_target, batch_size=32, epochs=20, validation_split=0.2)

print(logits_to_text(encodeco_model.predict(tmp_x[:1])[0], target_token))

ValueError: ignored

Original

In [ ]:
encodeco_model.save('/content/gdrive/MyDrive/Colab_Notebooks/NLP/translation/encode')

Based on code from https://towardsdatascience.com/neural-machine-translation-with-python-c2f0a34f7dd

In [ ]:
# Train Embedding Bi-directional model

tmp_X = pad(pre_source)
model = model_final(tmp_X.shape,
                    pre_target.shape[1],
                    len(source_token.word_index)+1,
                    len(target_token.word_index)+1)

model.fit(tmp_X, pre_target, batch_size = 32, epochs = 17, validation_split = 0.2)

Epoch 1/17
1130/1130 [==============================] - 471s 412ms/step - loss: 0.6875 - accuracy: 0.9180 - val_loss: 0.5706 - val_accuracy: 0.9284
Epoch 2/17
1130/1130 [==============================] - 466s 412ms/step - loss: 0.6538 - accuracy: 0.9200 - val_loss: 0.5681 - val_accuracy: 0.9287
Epoch 3/17
1130/1130 [==============================] - 466s 412ms/step - loss: 0.6552 - accuracy: 0.9203 - val_loss: 0.5678 - val_accuracy: 0.9294
Epoch 4/17
1130/1130 [==============================] - 467s 413ms/step - loss: 0.6493 - accuracy: 0.9211 - val_loss: 0.5636 - val_accuracy: 0.9303
Epoch 5/17
1130/1130 [==============================] - 467s 413ms/step - loss: 0.6433 - accuracy: 0.9219 - val_loss: 0.5591 - val_accuracy: 0.9317
Epoch 6/17
1130/1130 [==============================] - 468s 414ms/step - loss: 0.6371 - accuracy: 0.9228 - val_loss: 0.5566 - val_accuracy: 0.9319
Epoch 7/17
1130/1130 [==============================] - 467s 414ms/step - loss: 0.6282 - accuracy: 0.9236 - val_

In [ ]:
model.save('/content/gdrive/MyDrive/Colab_Notebooks/NLP/translation/bidi-encode')